In [1]:
import requests
from bs4 import BeautifulSoup
import re
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import matplotlib.pyplot as plt

In [2]:
import numpy as np
#import pandas as pd
#import re
import os
import codecs
#from sklearn import feature_extraction
import mpld3
import sys
import simplejson
import json
import pickle
import os

In [3]:
#Doc2Vec/Word2vec packages
import nltk
from nltk.tokenize import sent_tokenize
import gensim
from gensim import corpora, models, similarities
import logging
import os
import wikipedia
import random
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec

/Users/markregalla/anaconda/lib/python2.7/site-packages/numpy/lib/utils.py:95: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)
/Users/markregalla/anaconda/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/Users/markregalla/anaconda/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/Users/markregalla/anaconda/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/Users/markregalla/anaconda/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is depr

###Loop through many wikipedia pages
Runs them through training model

Wikipedia API guide:
https://en.wikipedia.org/wiki/Special:ApiSandbox#action=query&prop=info&format=json&inprop=url&pageids=1000005

In [80]:
%%time
i = 44962         #i = 44962  (12-4-15 11:43 pm)
counter = 9103   #counter = 9103 (12-4-15 11:43 pm)
#good_id_list = [] #be sure only to initialize as empty list the first time
#id_list = []      #use new list if you want to preserve good_id_list in memory

while counter < 50000:   #number is limit of number of page ids to get
    scrape_worked = False
    #if i == 1000:
    #    print 'went on too long'
    #    break
    page_id = 1000000 + i
    
    #try to scrape webpage.  Ignore if bad page error is returned from wikipedia
    try:
        r = requests.get('https://en.wikipedia.org/w/api.php?action=query&prop=info\
                &format=json&inprop=url&pageids=' + str(page_id))
        rtext = r.text
        jsontext = json.loads(rtext)
        if jsontext['query']['pages'][str(page_id)]['ns'] == 0:
            scrape_worked = True
    except:
        i += 1    
    
    if scrape_worked == True:
        article = requests.get('https://en.wikipedia.org/?curid=' + str(page_id))
        soup = BeautifulSoup(article.text, 'html.parser')
        paragraphs = soup.findAll('p')
        if len(paragraphs) >= 10:
            good_id_list.append(page_id)
            #id_list.append(page_id)    #for preservation of good_id_list
            corpus = page_read(soup)
            
            #write each wikipedia article to its own file
            with open("/users/markregalla/desktop/metis/Project4/WikiArticles/page_" 
                      + str(page_id) + ".txt", "w") as myfile:
                myfile.write(corpus.encode("UTF-8"))
            myfile.close
            counter += 1
    i += 1
    
    #prints iterators as a status
    sys.stdout.write("\r" + 'Attempts: ' + str(i) + '  Sucesses: ' + str(counter))   

print '\n' + str(len(good_id_list))
#print '\n' + str(len(id_list))

Attempts: 259987  Sucesses: 50000
50000
CPU times: user 3h 23min 41s, sys: 8min 50s, total: 3h 32min 32s
Wall time: 14h 10min 12s


In [22]:
#dump list of good id nos into pickle file
with open('good_id_list_4.pkl', 'w') as picklefile:
    pickle.dump(good_id_list, picklefile)

In [5]:
#retrieve list of good id nos
with open('good_id_list_1.pkl', 'r') as picklefile:
    good_id_list = pickle.load(picklefile)

In [38]:
#test scraping and appending to file
useid = 1000076
article = requests.get('https://en.wikipedia.org/?curid=1000076')# + str(useid))
#article = requests.get('https://en.wikipedia.org/wiki/William_Cleland')
soup = BeautifulSoup(article.text, 'html.parser')
paragraphs = soup.findAll('p')
print len(paragraphs)
#corpus = page_read(soup)
#f = open('output.txt', 'w')
#simplejson.dump(corpus, f)
#f.close()
#with open("total_corpus.txt", "ab") as myfile:
#    myfile.write(corpus)

5


In [130]:
#make webpage request #for next box
#r = requests.get('https://en.wikipedia.org/w/api.php?action=query&meta=siteinfo&\
#                 format=json&siprop=namespaces&pageids=1200008')
r = requests.get('https://en.wikipedia.org/w/api.php?action=query&prop=info\
                &format=json&inprop=url&pageids=1000001')#   + str(useid))
#figure out what part of the returned json has the namespace data
rtext = r.text
jsontext = json.loads(rtext)
print jsontext['query']['pages']['1000001']['ns']
jsontext

In [31]:
#Get article names from saved files in directory
#good_id_list = []
#for article in os.listdir('/users/markregalla/desktop/metis/Project4/WikiArticles'):
#    good_id_list.append(article[5:12])
good_id_list[0]    

'1000003'

In [59]:
#url = ['https://en.wikipedia.org/wiki/Myocardial_infarction',
#      'https://en.wikipedia.org/wiki/Blood_flow',
#      'https://en.wikipedia.org/wiki/Circulatory_system#Human_cardiovascular_system']

#TaggedDocument = gensim.models.doc2vec.TaggedDocument
u = 0
model_list = []
paralist = []

#while loop for scraping pages for training
#while u < 2:

#for loop for opening each file in directory for training
for article in os.listdir('/users/markregalla/desktop/metis/Project4/WikiArticles'):
    
    #Scrape page to get data for training
    #response = requests.get('https://en.wikipedia.org/?curid=' + str(good_id_list[u]))
    #page = response.text
    #soup = BeautifulSoup(page, 'html.parser')
    #para = page_read(soup)
    
    #Open saved total_corpus file for training
    with open("/users/markregalla/desktop/metis/Project4/WikiArticles/" + article, "r") as myfile:
        para = myfile.read()
    myfile.close
    
    #print para     #print the text being modeled
    #para = cleanText(para)
    #para = labelizeReviews(para, 'Testing')
    #paralist = paralist + para
    paralist.append(para)

    #Model articles individually as they are read or scraped in
    #model = gensim.models.Doc2Vec(para, size=200, window=8, min_count=5, workers=4)
    #model_list.append(model)
    #print model.docvecs['Testing_' + str(u)]
    #model.build_vocab(para)
    #model.sort_vocab()
    #model.train(para)
    #print model
    #print model.estimate_memory()
    
    #sys.stdout.write("\r" + str(u))   #prints iterator as a status
    u += 1
    if u == 20:
        break
#print paralist[0]
print len(paralist)

20


In [60]:
print type(paralist)
paralist = cleanText(paralist)
#paralist = labelizeReviews(paralist, 'Testing')
paralist[0]

<type 'list'>


'an ettercap is a race of bestial spider-men aberrations in the dungeons & dragons fantasy roleplaying game . \n\n\nettercaps first appeared in the original first edition ad&d fiend folio .  [ 1 ]  these original ettercaps had human-like faces ,  mouths ,  hands and feet .  these ettercaps were also covered by a layer of short hair and had large noses . \nthis edition of the d&d game included its own version of the ettercap ,  in the dungeons & dragons adventure game set  ( 1999 )  .  [ 2 ] \nthe ettercap appeared first in the monstrous compendium volume two  ( 1989 )  ,  [ 3 ]  and was reprinted in the monstrous manual  ( 1993 )  .  [ 4 ]  the ettercaps of second edition retained the hairy bodies and human mouths/hands and feet of their predecessors .  these ettercaps were described as having reptilian eyes that were usually of a reddish hue .  they were colored of a splotchy brown-grey mix ,  with an orange furred underbelly . \nthe ettercap appeared in the monster manual for this ed

In [39]:
cleaned_para = []
for para in paralist:
    cleaned_para.append(cleanText(para))
cleaned_para[0]

[['a'],
 ['n'],
 [],
 ['e'],
 ['t'],
 ['t'],
 ['e'],
 ['r'],
 ['c'],
 ['a'],
 ['p'],
 [],
 ['i'],
 ['s'],
 [],
 ['a'],
 [],
 ['r'],
 ['a'],
 ['c'],
 ['e'],
 [],
 ['o'],
 ['f'],
 [],
 ['b'],
 ['e'],
 ['s'],
 ['t'],
 ['i'],
 ['a'],
 ['l'],
 [],
 ['s'],
 ['p'],
 ['i'],
 ['d'],
 ['e'],
 ['r'],
 ['-'],
 ['m'],
 ['e'],
 ['n'],
 [],
 ['a'],
 ['b'],
 ['e'],
 ['r'],
 ['r'],
 ['a'],
 ['t'],
 ['i'],
 ['o'],
 ['n'],
 ['s'],
 [],
 ['i'],
 ['n'],
 [],
 ['t'],
 ['h'],
 ['e'],
 [],
 ['d'],
 ['u'],
 ['n'],
 ['g'],
 ['e'],
 ['o'],
 ['n'],
 ['s'],
 [],
 ['&'],
 [],
 ['d'],
 ['r'],
 ['a'],
 ['g'],
 ['o'],
 ['n'],
 ['s'],
 [],
 ['f'],
 ['a'],
 ['n'],
 ['t'],
 ['a'],
 ['s'],
 ['y'],
 [],
 ['r'],
 ['o'],
 ['l'],
 ['e'],
 ['p'],
 ['l'],
 ['a'],
 ['y'],
 ['i'],
 ['n'],
 ['g'],
 [],
 ['g'],
 ['a'],
 ['m'],
 ['e'],
 ['.'],
 [],
 [],
 [],
 ['e'],
 ['t'],
 ['t'],
 ['e'],
 ['r'],
 ['c'],
 ['a'],
 ['p'],
 ['s'],
 [],
 ['f'],
 ['i'],
 ['r'],
 ['s'],
 ['t'],
 [],
 ['a'],
 ['p'],
 ['p'],
 ['e'],
 ['a'],
 ['r'],
 ['e'],

In [37]:
all_model = gensim.models.Doc2Vec(paralist, size=200, window=8, min_count=5, workers=4)
print all_model.docvecs['Testing_0']

[[[ -2.08980720e-02  -1.97894238e-02   2.51624659e-02   4.59751859e-03
     4.57690582e-02  -3.03693898e-02  -2.97904201e-02  -5.80354296e-02
    -2.54040100e-02  -3.94951999e-02   6.43454567e-02  -5.49058691e-02
     3.33979279e-02  -3.55769098e-02  -2.76284711e-03  -5.58371544e-02
    -4.51013185e-02   2.74869166e-02   2.64361743e-02   3.93223315e-02
    -2.43464522e-02  -1.81727123e-03   4.93163988e-02  -4.62287478e-02
     6.21454827e-02  -2.18997803e-02   4.18519601e-03   2.17317864e-02
     3.44078504e-02  -2.51301564e-02  -5.09813726e-02   5.69769740e-02
     2.55317912e-02   1.32764224e-02  -3.54596712e-02  -5.78556210e-02
     5.17803319e-02  -4.88181487e-02   6.32253662e-02  -1.30228736e-02
    -2.02839468e-02  -6.26325160e-02  -3.45881470e-03   8.48615170e-03
    -8.38861894e-03  -1.45415245e-02   2.51015592e-02   2.09790394e-02
     4.22517993e-02  -4.30729240e-02   1.48973009e-02   5.02680950e-02
    -9.51159466e-03   3.86138149e-02   4.23070006e-02   1.19440472e-02
     4

In [38]:
concat_model = ConcatenatedDoc2Vec([x for x in model_list])
print concat_model.docvecs['Testing_0']

[[[ -1.83365634e-03  -1.73638144e-03   2.20782799e-03   4.03400016e-04
     4.01591184e-03  -2.66468967e-03  -2.61389744e-03  -5.09220362e-03
    -2.22901767e-03  -3.46541638e-03   5.64583903e-03  -4.81759524e-03
     2.93042324e-03  -3.12161539e-03  -2.42420778e-04  -4.89932671e-03
    -3.95731814e-03   2.41178460e-03   2.31958716e-03   3.45026120e-03
    -2.13622325e-03  -1.59452960e-04   4.32716729e-03  -4.05623158e-03
     5.45282476e-03  -1.92154921e-03   3.67220986e-04   1.90681440e-03
     3.01903603e-03  -2.20498885e-03  -4.47325176e-03   4.99932840e-03
     2.24023149e-03   1.16491434e-03  -3.11132590e-03  -5.07642794e-03
     4.54335939e-03  -4.28345101e-03   5.54758357e-03  -1.14266691e-03
    -1.77977374e-03  -5.49554173e-03  -3.03486537e-04   7.44599616e-04
    -7.36039481e-04  -1.27591437e-03   2.20247987e-03   1.84075686e-03
     3.70729668e-03  -3.77933099e-03   1.30712963e-03   4.41066828e-03
    -8.34574574e-04   3.38809285e-03   3.71214864e-03   1.04800356e-03
     4

In [41]:
#find difference bewteen instantiating model, then building model
model = gensim.models.Doc2Vec(size=200, window=8, min_count=5, workers=4)
model.build_vocab(para)
print model
print model.estimate_memory()

Doc2Vec(dm/s,d200,hs,w8,mc5,t4)
{'vocab': 7700, 'doctag_lookup': 200, 'doctag_syn0': 800, 'total': 26300, 'syn0': 8800, 'syn1': 8800}


In [42]:
#or just running text through gensim model
model = gensim.models.Doc2Vec(para, size=200, window=8, min_count=5, workers=4)
print model
print model.estimate_memory()

Doc2Vec(dm/s,d200,hs,w8,mc5,t4)
{'vocab': 7700, 'doctag_lookup': 200, 'doctag_syn0': 800, 'total': 26300, 'syn0': 8800, 'syn1': 8800}


In [4]:
def page_read(soup):
    #thetext = []
    thetext = ''
    paragraphs = soup.findAll('p') #comment out only for when saving wiki articles to disk
    for p in paragraphs:
        #thetext.append(p.text)    #make a list of sentences
        thetext = thetext + p.text + '\n' #make a list with 1 element being the whole document
    #thetext = [thetext]
    #theindex = 0
    #wikidef = paragraphs[theindex].text
    #print thetext
    #print type(thetext)
    return thetext

In [58]:
def cleanText(corpus):
    punctuation = """.,?!:;(){}[]"""
    corpus = [z.lower().replace('[*]','') for z in corpus]   #

    #treat punctuation as individual words
    for c in punctuation:
        corpus = [z.replace(c, ' %s '%c) for z in corpus]
    #corpus = [z.split() for z in corpus]
    return corpus

In [6]:
def labelizeReviews(reviews, label_type):
    labelized = []
    print len(reviews)
    for i,v in enumerate(reviews):
        #label = '%s_%s'%(label_type,i)    #iterate sentence labels
        label = '%s_%s'%(label_type,1)     #uniform sentence labels
        labelized.append(TaggedDocument(v, [label]))
    return labelized
    #yield LabeledSentence(words=line.split(), labels=['SENT_%s' % uid])
        #yield gensim.models.doc2vec.LabeledSentence(words=line.split(), tags=['SENT_%s' % uid])

In [90]:
doc1=["This is a sentence","This is another sentence"]
documents=[doc.strip().split(" ") for doc in doc1 ]
model = gensim.models.Doc2Vec(documents, size = 100, window = 300, min_count = 10, workers=4)

AttributeError: 'list' object has no attribute 'words'

In [93]:
documents = ["Human machine interface for lab abc computer applications",
            "A survey of user opinion of computer system response time",
            "The EPS user interface management system",
            "System and human system engineering testing of EPS",
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
        for document in documents]
model = gensim.models.Doc2Vec(texts, size = 100, window = 300, min_count = 10, workers=4)

AttributeError: 'list' object has no attribute 'words'